This notebook set's up an FEM simulation by sampling from the analytical approximation of the surface patch with some $\theta$. Then we translate this into a simnibs input matrix and run a simulation and return the results

In [8]:
import sys

In [9]:
#SCC package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/Python_modules/src')

In [10]:
import numpy as np
from simnibs import sim_struct, run_simulation

#For importing geometry library
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

#Timing
from timeit import default_timer as timer

In [2]:
from fieldopt import geolib, tetralib

ImportError: No module named simnibs.msh

In [12]:
#Load up the geolibrary as a module
glib_path = '/projects/jjeyachandra/rtms_optimize/notebooks/python2/'
nbload = NotebookLoader(glib_path)
geolib = nbload.load_module('geolib')

importing Jupyter notebook from /projects/jjeyachandra/rtms_optimize/notebooks/python2/geolib.ipynb


In [13]:
#Load up quadratic surface constants, inverse rotation matrix, bounds and normal vector
#This is for the Bayesian optimization component
C = np.fromfile('../../output/quadratic_vec')
iR = np.fromfile('../../output/inverse_rot').reshape((3,3))
b  = np.fromfile('../../output/param_bounds').reshape((3,2))

### Get raw input coordinates that need to be transformed

In [14]:
#Sample a coordinate within the bounds and form input array
x = np.random.uniform(b[0,0], b[0,1])
y = np.random.uniform(b[1,0], b[1,1])

#Sample rotation
r = np.random.uniform(0,180)
print(x,y,r)

(28.015849250971648, 24.65804834558192, 77.22623010750557)


In [15]:
#Compute z value then map to mesh surface
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Compute orientation vector in basis of surface normal then rotate into FEM mesh space
preaff_rot, preaff_n = geolib.map_rot_2_surf(x,y,r,C)
rot = np.matmul(iR,preaff_rot)
n = np.matmul(iR,preaff_n)

In [16]:
#Start a session
S = sim_struct.SESSION()
S.fnamehead="../../data/simnibs_output/sub-CMH090.msh"
S.pathfem="../../data/simnibs_output/test_simulation"

In [17]:
#Define the TMS simulation
tms = S.add_tmslist()
tms.fnamecoil='/projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz'

In [18]:
#Define position
pos = tms.add_position()
pos.matsimnibs = geolib.define_coil_orientation(loc,rot,n)

In [ ]:
start = timer()
run_simulation(S)
stop = timer()
print(stop - start)